# Starter's Guide to building a CNN with keras (TF), and google drive for MNIST recognition

This guide is meant to be a simple tutorial which lets you take images classes, train a model with keras and then predict a new image with your "intelligent" model.

To have this notebook work you will need to:

Have a google drive folder; copy this notebook to it

To run this notebook select each cell and click shift+enter on your keyboard, wait until the cell is finished running before going to the next

Below outline the standard import and installation settings for Keras

In [0]:
## STEP 1

# #This section is base copy from google's sample import of tensorflow, keras
!pip install -q keras

In [0]:
## STEP 2 
#Here we connect to your google drive folder.
# And Generate auth tokens for Colab

# CLICK THE LINK GENERATED THEN COPY AND PASTE THE KEY INTO THE FIELD AND CLICK ENTER
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# print 'Files in Drive:'
!ls "/content/drive/My Drive/"

In [0]:
#Is GPU Working?
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
#Which GPU Am I Using?
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

# **0-IMPORTING LIBRARIES**

In [0]:
# Importing the Keras libraries and packages
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras import optimizers
import numpy as np
import matplotlib.pyplot as plt

# **1- LOAD DATA** 

In [0]:
# Let's load the MNIST dataset.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Plot handwritten digits 
r = np.random.randint(x_train.shape[0])
plt.imshow(x_train[r],cmap = "gray")

In [0]:
# # Reshape data to fit model
X_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
X_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [0]:
# Conversion to float32 & Normalisation DATA
X_train= X_train.astype('float32')
X_test= X_test.astype('float32')
X_train = X_train/255
X_test = X_test/255

print()
print('MNIST data loaded: train:',len(x_train),'test:',len(x_test))
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)

In [0]:
# # one-hot encoding:
Y_train_cat = to_categorical(y_train,num_classes = 10)
Y_test_cat = to_categorical(y_test,num_classes = 10)

## **2-DEFINE CNN KERAS Model**

In [0]:
# Initialising the CNN

model = Sequential()
# Step 1 - Convolution
# make 64 feature detectors with a size of 3x3
# choose the input-image's format to be 28x28 with 1 channels
model.add(Conv2D(64, (3, 3), input_shape=(28, 28, 1), padding='same', activation='relu'))
# Step 2 - Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# Adding a second convolutional layer
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# Step 3 - Flattening
model.add(Flatten()) #Passage de tableau 2D à 1D
# Step 4 - Full connection
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))


In [0]:
# Affichage de la structure du réseau dans la console
print(model.summary())

# **3-COMPILE KERAS MODEL**

In [0]:
# Optimizer et loss function
model.compile(loss = "categorical_crossentropy",optimizer = "adam", metrics = ['accuracy'])

## **4-FIT KERAS MODEL --Learning**

In [0]:
# Apprentissage du modèle
hist = model.fit(X_train,Y_train_cat,epochs = 10,batch_size = 128,validation_data=(X_test, Y_test_cat))

In [0]:
# Save model
#Keras separates the concerns of saving your model architecture and saving your model weights.
#Model weights are saved to HDF5 format. This is a grid format that is ideal for storing multi-dimensional arrays of numbers.

model.save("/content/drive/My Drive/code workshop/model_mnist1.h5")
model.save_weights("/content/drive/My Drive/code workshop/weightmodel_mnist1.h5")
print("Saved model and weights")

Saved model and weights


In [0]:
# Print learning history 
figure = plt.figure()

ax1 = plt.subplot(121)
ax1.plot(hist.history["loss"],label = "Training Loss")
plt.xlabel("Nombre d'itérations")
plt.ylabel("Loss function")
plt.legend()

ax2 = plt.subplot(122)
ax2.plot(hist.history["acc"],label = "Training Accuracy")
plt.xlabel("Nombre d'itérations")
plt.ylabel("Accuracy")
plt.legend()


# **5-EVALUATE KERAS MODEL**

In [0]:
score = model.evaluate(X_test, Y_test_cat,batch_size = 128)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# **6-MAKE PREDICTION**

In [0]:
prediction = model.predict(X_test)
prediction = np.argmax(prediction,axis = 1)

print("Accuracy on test set: " + str(np.round(np.sum(prediction == y_test)/x_test.shape[0]*100,3)) + " %")

# **Visualize Model Training History in Keras**

In [0]:
#Graphiques  et  résultats
history_dict = hist.history
history_dict.keys()
# dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc = history_dict['acc']
val_acc = history_dict['val_acc'] #ce qui nous intéresse
epochs = range(1, len(acc) + 1)
#perte
plt.plot(epochs, loss_values, 'r-', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig("/content/drive/My Drive/workshop/CNN-Loss.png", bbox_inches="tight", dpi=600)
plt.show()
#précision
plt.clf()
plt.plot(epochs, acc, 'r-', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy FNN')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("/content/drive/My Drive/workshop/CNN-Accuracy.png", bbox_inches="tight", dpi=600)
plt.show()  #!!!!   affiche et remet à zéro => sauvegarder avant 
 
